In [1]:
import torch
from torch.utils.data import DataLoader
from object_detection_data import Encoder, TransformedDataset
from pytorch_lightning.callbacks import RichProgressBar
import pytorch_lightning as pl
from object_detection_models import MultiClassJetNet

torch.manual_seed(2)


In [2]:
default_scalings = torch.tensor(
    [[0.25, 0.25], [0.2, 0.2], [0.125, 0.125], [0.125 / 2, 0.125 / 2], [0.125 / 4, 0.125 / 4]])
feature_map_size = (8, 10)
num_classes = 4
encoder = Encoder(default_scalings, feature_map_size, num_classes)
transformed_train_data = TransformedDataset('data/train', encoder)
transformed_val_data = TransformedDataset('data/val', encoder)


In [3]:
train_loader = DataLoader(transformed_train_data, batch_size=4,
                          shuffle=True, num_workers=4)
val_loader = DataLoader(transformed_val_data, batch_size=4,
                        shuffle=False, num_workers=4)

pl_model = MultiClassJetNet(num_classes, len(default_scalings))
trainer = pl.Trainer(limit_predict_batches=100,
                     max_epochs=20)
trainer.fit(model=pl_model, train_dataloaders=train_loader,
            val_dataloaders=val_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

   | Name                   | Type               | Params
---------------------------------------------------------------
0  | accuracy               | MulticlassAccuracy | 0     
1  | batch_normalization_1  | BatchNorm2d        | 2     
2  | conv2d_1               | Conv2d             | 160   
3  | batch_normalization_2  | BatchNorm2d        | 32    
4  | depthwise_conv2d_1     | Conv2d             | 144   
5  | conv2d_2               | Conv2d             | 408   
6  | batch_normalization_3  | BatchNorm2d        | 48    
7  | depthwise_conv2d_2     | Conv2d             | 216   
8  | conv2d_3               | Conv2d             | 400   
9  | batch_normalization_4  | BatchNorm2d        | 32    
10 | depthwise_conv2d_3     | Conv2d             | 144   
11 | conv2d_4               | Conv2d             | 340   
12 | batch_normalization_5  | Batc

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/jonathan/hulks/ml/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
